In [ ]:
import pandas as pd        
import numpy as np       
                            
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler   
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [ ]:
xlsx1_filepath = r"F:\Newdataset\norm_ALL_CC_Delay.xlsx"

data_1 = pd.read_excel(xlsx1_filepath)

x = data_1[data_1.columns[1:]]
y = data_1['grade']
colNames = x.columns
x = x.astype(np.float64)
x = StandardScaler().fit_transform(x) 
X1 = pd.DataFrame(x)
X1.columns = colNames

## MI

In [ ]:
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
mi = SelectKBest(mutual_info_classif,k = 200).fit(X1,y)
len(X1.columns[mi.get_support()])
X_mi=mi.transform(X1)

X_mi = pd.DataFrame(X_mi)
X_mi.columns = X1.columns[mi.get_support()]

# X=X[features]

## RFE

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(RandomForestClassifier(n_estimators=100,random_state=0),n_features_to_select=100,step=10)
rfe.fit(X_mi,y)

X_features = X_mi.columns[rfe.get_support()]

X1=X1[X_features]

# X_RFE=rfe.transform(X_selected)

## LASSO

In [ ]:
%%time
import warnings
warnings.filterwarnings("ignore")

alphas = np.logspace(-5,2,100) 
# print(alphas)
model_lassoCV = LassoCV(alphas = alphas, cv = 10, max_iter = 100000).fit(X1,y) 
print('The Optimal alpha is :',model_lassoCV.alpha_)  
print('--------------------')
coef = pd.Series(model_lassoCV.coef_ ,index = X1.columns)
# print(coef) 
print('LASSO selected ' + str(sum(coef != 0)) + ' variable and eliminated the other' +  str(sum(coef == 0)))

In [ ]:
index = coef[coef != 0 ].index
X1 = X1[index]
print(coef[coef != 0])

## Convolution features

In [ ]:
xlsx2_filepath = r"F:\Newdataset\muilt_ROI\CC_deep_features2048.xlsx"

data_2 = pd.read_excel(xlsx2_filepath)
x = data_2[data_2.columns[1:]]
y = data_2['grade']
colNames = x.columns
x = x.astype(np.float64)
x = StandardScaler().fit_transform(x) 
X2 = pd.DataFrame(x)
X2.columns = colNames

In [ ]:
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
mi = SelectKBest(mutual_info_classif,k = 100).fit(X2,y)
len(X2.columns[mi.get_support()])
X2_mi=mi.transform(X2)

X2_mi = pd.DataFrame(X2_mi)
X2_mi.columns = X2.columns[mi.get_support()]

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(RandomForestClassifier(n_estimators=100,random_state=0),n_features_to_select=5,step=10)
rfe.fit(X2_mi,y)

X2_features = X2_mi.columns[rfe.get_support()]

X2=X2[X2_features]

In [ ]:
%%time

alphas = np.logspace(-6,2,100) 
# print(alphas)
model_lassoCV = LassoCV(alphas = alphas ,cv = 10, max_iter = 100000).fit(X2,y) 
print('The Optimal alpha is :',model_lassoCV.alpha_)  
print('--------------------')
coef = pd.Series(model_lassoCV.coef_ ,index = X2.columns)   
# print(coef)
print('LASSO selected ' + str(sum(coef != 0)) + ' variable and eliminated the other' +  str(sum(coef == 0)))

In [ ]:
index = coef[coef != 0 ].index
X2 = X2[index]
print(coef[coef != 0])

In [ ]:
Xs = pd.concat([X1,X2],axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (12,6),dpi = 300)
sns.heatmap(Xs.corr(),xticklabels = Xs.corr().columns,yticklabels = Xs.corr().columns,cmap = 'RdBu',center = 0,annot = True)

plt.title('Correlation coefficient matrix of features for E0',fontsize = 20)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show()

## SVM

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(Xs,y,test_size = 0.3)
model_svm = svm.SVC(kernel='rbf',gamma = 'auto',class_weight= 'balanced',probability = True).fit(x_train,y_train)
score_svm1 = model_svm.score(x_train,y_train)
score_svm2 = model_svm.score(x_test,y_test)
print(score_svm1)
print(score_svm2)

In [ ]:
from sklearn.model_selection import GridSearchCV
Cs = np.logspace(-2,6,50,base = 2)
gammas = np.logspace(-6,4,50,base = 2)
param_grid = dict(C = Cs,gamma = gammas)
grid = GridSearchCV(svm.SVC(kernel='rbf',class_weight= 'balanced'),param_grid = param_grid,cv = 5).fit(Xs,y)
print(grid.best_params_)
#'C': 0.6922608929229022, 'gamma': 0.055812601103520335

In [ ]:
from sklearn.model_selection import cross_val_score
C = grid.best_params_['C']
gamma = grid.best_params_['gamma']
x_train,x_test,y_train,y_test = train_test_split(Xs,y,test_size = 0.3)
model_svm = svm.SVC(C = C,kernel='rbf',gamma =gamma,class_weight= 'balanced',probability = True).fit(x_train,y_train)
score_svm_train = model_svm.score(x_train,y_train)
score_svm_test = model_svm.score(x_test,y_test)
print(score_svm_train)
print(score_svm_test)

model_cv = cross_val_score(model_svm,Xs,y,cv = 10).mean()
print(model_cv)

In [ ]:
y_pred = clf.predict(x_test)
print((y_test).values)
print(y_pred)

from sklearn.metrics import accuracy_score,precision_score, \
recall_score,f1_score,cohen_kappa_score
print('ACC：',
      accuracy_score(y_test,y_pred))
print('precision：',
      precision_score(y_test,y_pred))
print('recall：',
      recall_score(y_test,y_pred))
print('f1_score：',
      f1_score(y_test,y_pred))


from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score,auc
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))

y_probs = model_svm.predict_proba(x_test)  
# # print(y_probs)             
# # print(y_probs[:,1])
fpr,tpr,thresholds = roc_curve(y_test,y_probs[:,1],pos_label = 1)

auc_score = auc(fpr,tpr)
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (AUC = %0.2f)' % auc_score)
# plt.plot(fpr,tpr,marker = '')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC for PVE')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.legend(loc="lower right")
plt.show()